# Домашнее задание 3. Fine-Tuning модели BERT и анализ альтернативных архитектур в задаче классификации

**ФИО Студента: Кузнецов Кирилл Игоревич**

**Дата Выполнения: 01.10.2025**

---

### **Описание задания**

В этом задании вы реализуете эксперементальное сравнение классического трансформера (BERT) с современными альтернативными архитектурами (Mamba) на задаче классификации русскоязычных текстов. Проведете исследование trade-offs между качеством, скоростью и количеством обучаемых параметров для различных подходов к Fine-Tuning.
---

## **Установка и импорт библиотек**

In [1]:
# # # Установка PEFT для LoRA
# !pip install peft

# # # Установка зависимостей для Mamba
# !pip install causal-conv1d mamba-ssm


In [2]:
# !pip install evaluate

In [15]:
import os
import json
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import time
import warnings
from collections import defaultdict

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed
)
from transformers.modeling_outputs import SequenceClassifierOutput

from datasets import Dataset as HFDataset
import evaluate

from peft import (
    LoraConfig,
    TaskType,
    get_peft_model
)

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

---
## **Задание 1. Подготовка данных и базовой модели**

Используем полный датасет русскоязычных отзывов с Кинопоиска. Для упрощения задачи бинарной классификации удаляем нейтральные отзывы. Разбиваем данные на обучающую и тестовую выборки в соотношении 80/20.

Задачи:
1. Загрузите датасет отзывов Кинопоиска и соответствующий токенизатор для DeepPavlov/rubert-base-cased.


In [6]:
# Загружаем полный датасет
print("Загружаем полный датасет отзывов...")

# Используем датасет отзывов Кинопоиска
df_full = pd.read_json(
    "hf://datasets/blinoff/kinopoisk/kinopoisk.jsonl", lines=True)
df_full.head()

Загружаем полный датасет отзывов...


,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."


In [7]:
# Словарь результатов моделирования
results = defaultdict()

In [8]:
# Немного предобработаем первичный датасет

# Если строка начинается с \n, то возвращается подстрока без первого
# символа иначе возвращается сама строка без изменений. Так же пропускается пустое значение.

df_full['text'] = df_full['content'].apply(
    lambda x: x[1:] if isinstance(x, str) and x.startswith('\n') else x
)

# Убираем нейтральные для упрощения (собственно таково задание)
df_full = df_full[df_full['grade3'] != 'Neutral']

# Присваиваем бинарную метку
df_full['label'] = df_full['grade3'].apply(lambda x: 1 if x == 'Good' else 0)

# Оставляем только контент и метку класса
df = df_full[['text', 'label']]

# Разделение на train/validation/test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts,
    train_labels,
    test_size=0.2,
    random_state=42,
    stratify=train_labels
)

# Выбираем русскоязычную модель
MODEL_NAME = "DeepPavlov/rubert-base-cased"


# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Загружаем конфигурацию и модель
config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config
)


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Определим функции для токенизации, вычисления метрик и подсчета обучаемых параметров, которые будут использоваться для всех моделей.   

2. Подготовьте данные: создайте dataset-объекты для обучающей и тестовой выборок, токенизируйте тексты и подготовьте их к подаче в модель в соответствии с семинаром 1 данной дисциплины.  
3. Определите функцию для вычисления метрик Accuracy, F1-score.  




  

In [9]:
MAX_LENGTH = 256 # Ограничиваем длину для ускорения обучения и экономии памяти

# 2. Подготовьте данные:

# Функция токенизации
def tokenize_function(examples, max_lenght=MAX_LENGTH):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=max_lenght
    )


# Создаём HuggingFace datasets
train_dataset = HFDataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = HFDataset.from_dict({'text': val_texts, 'label': val_labels})
test_dataset = HFDataset.from_dict({'text': test_texts, 'label': test_labels})


# Применяем токенизацию ко всем датасетам
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 3. Определите функцию для вычисления метрик Accuracy, F1-score.

# Функция для вычисления метрик
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1']
    }

Map:   0%|          | 0/20489 [00:00<?, ? examples/s]

Map:   0%|          | 0/5123 [00:00<?, ? examples/s]

Map:   0%|          | 0/6403 [00:00<?, ? examples/s]

---
## **Задания 2 и 3. Baseline — Fine-Tuning BERT**

В качестве baseline используем русскоязычную модель `DeepPavlov/rubert-base-cased`. Мы рассмотрим два подхода: полный Fine-Tuning и эффективный Fine-Tuning с помощью LoRA.

Задачи:  
**BERT Full Fine-Tuning:**
1. Загрузите предобученную модель DeepPavlov/rubert-base-cased.
2. Настройте TrainingArguments для полного дообучения.
3. Обучите модель на полном обучающем наборе данных.
4. Оцените качество на тестовой выборке и зафиксируйте время обучения и количество обучаемых параметров.  

**BERT с LoRA или иным методом (Parameter-Efficient Fine-Tuning):**
1. Снова загрузите исходную модель DeepPavlov/rubert-base-cased.
2. Настройте LoraConfig, указав целевые модули (например, query, value).
3. Примените LoRA к модели с помощью get_peft_model.
4. Обучите параметро-эффективную модель.
5. Оцените ее качество, время обучения и количество обучаемых параметров.   
6. Сравните с результатами полного дообучения.

In [10]:
# Изначально пытался на локальной машине, потому много оптимизации под MPS
# но m2 air не смог и с оптимизацией... пришлось идти на колаб
# так что поддержка MPS в классах аргументов можно считать легаси
print(f"MPS доступен: {torch.backends.mps.is_available()}")
print(f"MPS построен: {torch.backends.mps.is_built()}")

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Используется MPS (Apple Silicon)")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Используется {device}")

MPS доступен: False
MPS построен: False
Используется cuda


In [13]:
print("--- 1. BERT: Full Fine-tuning ---")

# Настройка гиперпараметров полного fine-tuning и поведения процесса обучения с
# помощью Hugging Face TrainingArguments.
# Эти параметры управляют тем, как модель будет обучаться, сохраняться и оцениваться.
training_args = TrainingArguments(
    output_dir="./results_full_finetuning",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    push_to_hub=False,
    report_to="none",  # Отключаем wandb/tensorboard для простоты
    # Добавляем поддержку MPS
    use_mps_device=True if device.type == "mps" else False,
)

# Создаём модель для полного fine-tuning
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config
)

# Перемещаем модель на устройство
model = model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

# Подсчитываем количество обучаемых параметров в модели.
params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Обучение с замером времени
train_start_time = time.time()
train_result = trainer.train()
train_time = time.time() - train_start_time # Время, затраченное на предсказания и оценку


# Оценка на тестовой выборке с замером времени
eval_start_time = time.time()
test_results = trainer.evaluate(eval_dataset=test_dataset)
eval_time = time.time() - eval_start_time


# Сохраняем результаты
results["bert_fine_tuning"] = {
    "accuracy": test_results['eval_accuracy'],
    "f1": test_results['eval_f1'],
    "training_time": train_time,
    "evaluation_time": eval_time,
    "parameters":params_count
}

--- 1. BERT: Full Fine-tuning ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.483900,0.318928,0.898692,0.888963
2,0.290100,0.280360,0.918212,0.910782
3,0.188200,0.233744,0.930509,0.926748


In [ ]:
drive.mount('/content/drive')
# Создаем имя файла с временной меткой
timestamp = time.strftime("%Y%m%d-%H%M%S")
filename = f"/content/drive/MyDrive/training_results_{timestamp}.json"

# Сохраняем
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"Результаты сохранены в {filename}")

In [14]:
# Проверим что словарь дополнился
print(results)


defaultdict(None, {'bert_fine_tuning': {'accuracy': 0.9370607527721381, 'f1': 0.9340853185890277, 'training_time': 3067.9095413684845, 'evaluation_time': 83.83732986450195, 'parameters': 177854978}})


In [16]:
print("--- 2. BERT: Frozen layers ---")

def freeze_bert_layers(model, num_layers_to_freeze):
    """
    Замораживает первые num_layers_to_freeze слоёв BERT.
    """
    # Замораживаем embedding-слой (включая token, position и segment embeddings)
    # Этот слой отвечает за преобразование токенов в векторные представления.
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False

    # Замораживаем указанное количество encoder слоёв
    for i in range(num_layers_to_freeze):
        for param in model.bert.encoder.layer[i].parameters():
            param.requires_grad = False

    # Подсчёт общего и обучаемого числа параметров для оценки эффективности заморозки
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Заморожено слоёв: {num_layers_to_freeze} из {model.config.num_hidden_layers}")
    print(f"Всего параметров: {total_params:,}")
    print(f"Обучаемых параметров: {trainable_params:,}")
    print(f"Процент обучаемых: {100 * trainable_params / total_params:.2f}%")

    return model

# Загружаем ту же архитектуру BERT, что и в полном fine-tuning
model_frozen = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config
)

# Перемещаем модель на MPS перед заморозкой
model_frozen = model_frozen.to(device)

# Замораживаем первые 10 слоёв (из 12)
model_frozen = freeze_bert_layers(model_frozen, num_layers_to_freeze=10)

# Проверяем, какие слои заморожены
print("\nСтатус слоёв:")
for i, layer in enumerate(model_frozen.bert.encoder.layer):
    is_frozen = not any(p.requires_grad for p in layer.parameters())
    print(f"Layer {i}: {'Заморожен' if is_frozen else 'Тренируемый'}")

# Обучение с замороженными слоями и MPS
training_args_frozen = TrainingArguments(
    output_dir="./results_frozen",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    use_mps_device=True if device.type == "mps" else False,
)

trainer_frozen = Trainer(
    model=model_frozen,
    args=training_args_frozen,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

params_count_frozen = sum(p.numel() for p in model_frozen.parameters() if p.requires_grad)


# Обучение с замером времени
train_start_time_frozen = time.time()
train_result_frozen = trainer_frozen.train()
train_time_frozen = time.time() - train_start_time_frozen


# Оценка на тестовой выборке с замером времени
eval_start_time_frozen = time.time()
test_results_frozen = trainer_frozen.evaluate(eval_dataset=test_dataset)
eval_time_frozen = time.time() - eval_start_time_frozen


# Сохраняем результаты
results["bert_frozen_layers"] = {
    "accuracy": test_results_frozen['eval_accuracy'],
    "f1": test_results_frozen['eval_f1'],
    "training_time": train_time_frozen,
    "evaluation_time": eval_time_frozen,
    "parameters": params_count_frozen
}


--- 2. BERT: Frozen layers ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Заморожено слоёв: 10 из 12
Всего параметров: 177,854,978
Обучаемых параметров: 14,767,874
Процент обучаемых: 8.30%

Статус слоёв:
Layer 0: Заморожен
Layer 1: Заморожен
Layer 2: Заморожен
Layer 3: Заморожен
Layer 4: Заморожен
Layer 5: Заморожен
Layer 6: Заморожен
Layer 7: Заморожен
Layer 8: Заморожен
Layer 9: Заморожен
Layer 10: Тренируемый
Layer 11: Тренируемый


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.208800,0.180685,0.937732,0.934652
2,0.197000,0.164220,0.941636,0.940079
3,0.169600,0.172744,0.944173,0.943116


In [17]:
# Создаем имя файла с временной меткой
timestamp = time.strftime("%Y%m%d-%H%M%S")
filename = f"/content/drive/MyDrive/training_results_{timestamp}.json"

# Сохраняем
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"Результаты сохранены в {filename}")


Mounted at /content/drive
Результаты сохранены в /content/drive/MyDrive/training_results_20251001-084825.json


In [18]:
# Проверим что словарь дополнился
print(results)


defaultdict(None, {'bert_fine_tuning': {'accuracy': 0.9370607527721381, 'f1': 0.9340853185890277, 'training_time': 3067.9095413684845, 'evaluation_time': 83.83732986450195, 'parameters': 177854978}, 'bert_frozen_layers': {'accuracy': 0.9467437138841168, 'f1': 0.94610839371943, 'training_time': 1421.1404066085815, 'evaluation_time': 85.06224536895752, 'parameters': 14767874}})


In [19]:
print("\n--- 3. BERT: LoRA Fine-tuning ---")

# Конфигурация LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Задача классификации последовательностей
    r=8,  # Ранг матриц адаптации
    lora_alpha=32,  # Масштабирующий параметр
    lora_dropout=0.1,  # Dropout для LoRA слоёв
    target_modules=["query", "value"],  # К каким модулям применять LoRA
    bias="none",  # Не трогаем bias
)

# Создаём новую модель для LoRA
model_lora_base = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    config=config
)

# Применяем LoRA
model_lora = get_peft_model(model_lora_base, lora_config)

# Перемещаем модель на MPS устройство
model_lora = model_lora.to(device)

# Печатаем информацию о обучаемых параметрах
model_lora.print_trainable_parameters()

# Получаем количество обучаемых параметров
params_count_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)

# Обучение с LoRA
training_args_lora = TrainingArguments(
    output_dir="./results_lora",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=3e-4,  # Более высокий learning rate для LoRA
    report_to="none",
    use_mps_device=True if device.type == "mps" else False,
)

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)


# Обучение с замером времени
train_start_time_lora = time.time()
train_result_lora = trainer_lora.train()
train_time_lora = time.time() - train_start_time_lora

# Оценка с замером времени
eval_start_time_lora = time.time()
test_results_lora = trainer_lora.evaluate(eval_dataset=test_dataset)
eval_time_lora = time.time() - eval_start_time_lora

# Сохраняем результаты
results["LoRA"] = {
    "accuracy": test_results_lora['eval_accuracy'],
    "f1": test_results_lora['eval_f1'],
    "training_time": train_time_lora,
    "evaluation_time": eval_time_lora,
    "parameters": params_count_lora
}


--- 3. BERT: LoRA Fine-tuning ---


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 178,151,428 || trainable%: 0.1664


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.199100,0.176653,0.937146,0.932299
2,0.166000,0.163612,0.940660,0.940724
3,0.157400,0.169476,0.944173,0.943183


In [21]:
# Создаем имя файла с временной меткой
timestamp = time.strftime("%Y%m%d-%H%M%S")
filename = f"/content/drive/MyDrive/training_results_+lora_{timestamp}.json"

# Сохраняем
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"Результаты сохранены в {filename}")

Результаты сохранены в /content/drive/MyDrive/training_results_+lora_20251001-092622.json


In [20]:
# Проверим что словарь дополнился
print(results)


defaultdict(None, {'bert_fine_tuning': {'accuracy': 0.9370607527721381, 'f1': 0.9340853185890277, 'training_time': 3067.9095413684845, 'evaluation_time': 83.83732986450195, 'parameters': 177854978}, 'bert_frozen_layers': {'accuracy': 0.9467437138841168, 'f1': 0.94610839371943, 'training_time': 1421.1404066085815, 'evaluation_time': 85.06224536895752, 'parameters': 14767874}, 'LoRA': {'accuracy': 0.9512728408558488, 'f1': 0.9510146506091981, 'training_time': 2114.351830482483, 'evaluation_time': 89.95970010757446, 'parameters': 296450}})


---
## **Задание 4. Альтернативная архитектура — Mamba**

Mamba — это современная архитектура на основе пространств состояний (State Space Models), которая показывает высокую производительность и линейную сложность по длине последовательности. Используем небольшую предобученную модель `state-spaces/mamba-130m-hf`.

Архитектура еще не распространенная, поэтому нужно самостоятельно написать блок для классификатора последовательностей. Вы можете воспользоваться готовым кодом ниже для эксперимента, либо установить библиотеку https://github.com/getorca/mamba_for_sequence_classification, либо протестировать иные архитектуры с huggingface

Задачи:  
1. Загрузите предобученную модель Mamba или другую, подходящую для классификации текста (например, state-spaces/mamba-130m-hf).
2. Адаптируйте модель для задачи бинарной классификации (добавьте классификационную голову).
3. Настройте TrainingArguments и проведите Fine-Tuning модели Mamba.
Оцените ее итоговое качество, время обучения и количество параметров.


In [57]:
# --- 1. Создаем наш собственный класс для классификации --- (пришлось немного его починить)
class CustomMambaForSequenceClassification(nn.Module):
    def __init__(self, model_name="state-spaces/mamba-130m-hf", num_labels=2):
        super().__init__()
        self.num_labels = num_labels

        # Загружаем базовую модель Mamba (без головы для конкретной задачи)
        self.mamba = AutoModel.from_pretrained(model_name)

        # Получаем размер скрытого состояния из конфигурации модели
        hidden_size = self.mamba.config.hidden_size

        # Создаем голову для классификации — обычный линейный слой
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Прогоняем данные через базовую модель Mamba
        outputs = self.mamba(input_ids=input_ids, attention_mask=attention_mask)

        # Mamba возвращает last_hidden_state. Его форма: (batch_size, sequence_length, hidden_size)
        last_hidden_state = outputs.last_hidden_state

        # Для классификации берем скрытое состояние ПОСЛЕДНЕГО токена в последовательности
        # Это стандартная практика для авторегрессионных моделей, как Mamba
        cls_embedding = last_hidden_state[:, -1, :]

        # Прогоняем его через наш классификатор, чтобы получить логиты
        logits = self.classifier(cls_embedding)

        # Если переданы метки (labels), вычисляем loss
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # Возвращаем результат в формате, который понимает Trainer
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=None, # Mamba не использует Attention, поэтому None
        )


In [58]:
# Исправляем функцию токенизации
def create_tokenized_datasets(tokenizer, train_dataset, test_dataset, max_length=512):
    """
    Токенизирует тренировочный и тестовый датасеты для модели Mamba
    """
    def tokenize_function(examples):
        # Токенизируем текст
        tokenized = tokenizer(
            examples['text'],
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        # Добавляем метки
        tokenized['labels'] = examples['label']
        return tokenized

    # Токенизируем датасеты
    tokenized_train = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=train_dataset.column_names
    )
    tokenized_test = test_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=test_dataset.column_names
    )

    return tokenized_train, tokenized_test

MODEL_NAME_MAMBA = "state-spaces/mamba-130m-hf"
TOKENIZER_NAME_MAMBA = "EleutherAI/gpt-neox-20b"

# Загружаем токенизатор
tokenizer_mamba = AutoTokenizer.from_pretrained(TOKENIZER_NAME_MAMBA)
if tokenizer_mamba.pad_token is None:
    tokenizer_mamba.pad_token = tokenizer_mamba.eos_token
print("Токенизатор для Mamba успешно загружен.")


# Токенизируем датасеты
tokenized_train_mamba, tokenized_test_mamba = create_tokenized_datasets(tokenizer_mamba, train_dataset, test_dataset)

# --- 2. Инициализируем нашу кастомную модель ---
model_mamba = CustomMambaForSequenceClassification(model_name=MODEL_NAME_MAMBA, num_labels=2)
model_mamba.mamba.config.pad_token_id = tokenizer_mamba.pad_token_id


# Обучение
training_args_mamba = TrainingArguments(
    output_dir="./results/mamba_custom",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    logging_dir='./logs/mamba_custom',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer_mamba = Trainer(
    model=model_mamba,
    args=training_args_mamba,
    train_dataset=tokenized_train_mamba,
    eval_dataset=tokenized_test_mamba,  # используем токенизированный тестовый датасет
    tokenizer=tokenizer_mamba,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer_mamba)
)

# Получаем количество обучаемых параметров
params_count_mamba = sum(p.numel() for p in model_mamba.parameters() if p.requires_grad)

# Обучение с замером времени
train_start_time_mamba = time.time()
train_result_mamba = trainer_mamba.train()
train_time_mamba = time.time() - train_start_time_mamba

# Оценка с замером времени - используем токенизированный датасет
eval_start_time_mamba = time.time()
eval_results_mamba = trainer_mamba.evaluate(eval_dataset=tokenized_test_mamba)
eval_time_mamba = time.time() - eval_start_time_mamba

# Сохраняем результаты
results["Mamba"] = {
    "accuracy": eval_results_mamba['eval_accuracy'],
    "f1": eval_results_mamba['eval_f1'],
    "training_time": train_time_mamba,
    "evaluation_time": eval_time_mamba,
    "parameters": params_count_mamba
}

Токенизатор для Mamba успешно загружен.


Map:   0%|          | 0/20489 [00:00<?, ? examples/s]

Map:   0%|          | 0/6403 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.278400,0.264899,0.927690,0.924474


In [59]:
# Создаем имя файла с временной меткой
filename = "/content/drive/MyDrive/training_results_whole.json"

# Сохраняем
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(f"Результаты сохранены в {filename}")

Результаты сохранены в /content/drive/MyDrive/training_results_whole.json


In [60]:
# Проверим что словарь дополнился
print(results)


defaultdict(None, {'bert_fine_tuning': {'accuracy': 0.9370607527721381, 'f1': 0.9340853185890277, 'training_time': 3067.9095413684845, 'evaluation_time': 83.83732986450195, 'parameters': 177854978}, 'bert_frozen_layers': {'accuracy': 0.9467437138841168, 'f1': 0.94610839371943, 'training_time': 1421.1404066085815, 'evaluation_time': 85.06224536895752, 'parameters': 14767874}, 'LoRA': {'accuracy': 0.9512728408558488, 'f1': 0.9510146506091981, 'training_time': 2114.351830482483, 'evaluation_time': 89.95970010757446, 'parameters': 296450}, 'Mamba': {'accuracy': 0.9276901452444167, 'f1': 0.9244735010374192, 'training_time': 2634.4738233089447, 'evaluation_time': 250.02479600906372, 'parameters': 129136898}})


---
## **Задание 5. Сравнительный анализ и выводы**

Проведите сравнительный анализ подходов и сделайте выводы на основе проведенных эксперементов.

Задачи:  
1. Создайте сводную таблицу, в которой будут отражены все ключевые показатели для трех подходов:
- BERT Full Fine-Tuning
- BERT + LoRA
- Mamba
2. Сравните модели по следующим критериям:
- Качество: Accuracy и F1-score.
- Эффективность: время обучения и количество обучаемых параметров.

3. Сформулируйте развернутые выводы:
- Какой подход показал наилучшее качество?
- Насколько LoRA сокращает количество параметров и время обучения по сравнению с полным Fine-Tuning? Как это влияет на метрики?
- Как Mamba показывает себя в сравнении с BERT? В чем ее сильные и слабые стороны для данной задачи?

4. Дайте рекомендации по выбору архитектуры в зависимости от ограничений (время, вычислительные ресурсы, требования к качеству).


In [61]:
# Открываем бэкап
filename = "/content/drive/MyDrive/training_results_whole.json"
with open(filename, encoding='utf-8') as f:
    result_json = json.load(f)

result_df = pd.DataFrame(result_json).T

# Находим ключевые метрики
min_time = result_df["training_time"].min()
max_accuracy = result_df["accuracy"].max()
max_f1 = result_df["f1"].max()

# Создаем индикаторные столбцы
# Помечаем строки с минимальным временем обучения
result_df["min_time"] = result_df["training_time"].apply(lambda x: "yes" if x == min_time else "no")

# Помечаем строки с лучшей точностью
result_df["best_accuracy"] = result_df["accuracy"].apply(lambda x: "yes" if x == max_accuracy else "no")

# Помечаем строки с лучшей F1-мерой
result_df["best_f1"] = result_df["f1"].apply(lambda x: "yes" if x == max_f1 else "no")

# Рассчитываем отклонения
result_df["best_accuracy_value"] = max_accuracy
result_df["best_f1_value"] = max_f1

# Разница от лучших значений
result_df["accuracy_diff_from_best"] = result_df["accuracy"] - max_accuracy
result_df["f1_diff_from_best"] = result_df["f1"] - max_f1

display(result_df)

result_df = result_df.reset_index()
result_df.to_csv("/content/drive/MyDrive/results_whole_df.csv", index=False)


,accuracy,f1,training_time,evaluation_time,parameters,min_time,best_accuracy,best_f1,best_accuracy_value,best_f1_value,accuracy_diff_from_best,f1_diff_from_best
bert_fine_tuning,0.937061,0.934085,3067.909541,83.837330,177854978.0,no,no,no,0.951273,0.951015,-0.014212,-0.016929
bert_frozen_layers,0.946744,0.946108,1421.140407,85.062245,14767874.0,yes,no,no,0.951273,0.951015,-0.004529,-0.004906
LoRA,0.951273,0.951015,2114.351830,89.959700,296450.0,no,yes,yes,0.951273,0.951015,0.000000,0.000000
Mamba,0.927690,0.924474,2634.473823,250.024796,129136898.0,no,no,no,0.951273,0.951015,-0.023583,-0.026541


## Сравнение по ключевым критериям

### 1. Качество (Accuracy и F1-score)
- Лучшее качество показала модель LoRA:  
  - Accuracy = 0.9513, F1 = 0.9510 — наивысшие значения среди всех подходов.
- Второе место — BERT с замороженными слоями (0.9467 / 0.9461), что интересно: даже без полного дообучения BERT может превзойти full fine-tuning.
- Full Fine-Tuning BERT показал худший результат среди BERT-вариантов (0.9371), что может указывать на переобучение или неоптимальную настройку гиперпараметров.
- Mamba оказалась наименее точной (0.9277 / 0.9245), уступая даже базовому BERT.

### 2. Эффективность

#### Время обучения
- Самое быстрое обучение — у BERT с замороженными слоями (1421 с).
- LoRA обучалась дольше (2114 с), но всё ещё быстрее, чем full fine-tuning BERT (3068 с).
- Mamba требует значительных вычислительных затрат (2634 с), почти столько же, сколько full BERT, но при этом даёт худшее качество.

#### Количество обучаемых параметров
- LoRA — лидер по эффективности: всего 296 тыс. обучаемых параметров.
  - Это в 600 раз меньше, чем у full BERT (177 млн).
  - И в 50 раз меньше, чем у BERT с замороженными слоями (14.7 млн).
- Mamba имеет 129 млн параметров — меньше, чем BERT (177 млн), но всё ещё очень много, особенно учитывая её более низкое качество.

---

## Выводы

### 1. Подход продемонсрировавший лучшее качество
LoRA продемонстрировала наилучшее качество по обоим метрикам (Accuracy и F1). Это примечательно, поскольку она обучает лишь небольшую долю параметров, но при этом превосходит как полный fine-tuning, так и другие упрощённые стратегии. Это говорит о высокой эффективности адаптации через низкоранговые матрицы.

### 2. Насколько LoRA сокращает ресурсы и как это влияет на метрики?
- Параметры: LoRA сокращает число обучаемых параметров с 177 млн до 0.3 млн.
- Время обучения: сокращается по сравнению с full fine-tuning.
- Влияние на качество: несмотря на радикальное сокращение параметров, качество улучшилось, а не ухудшилось. Это свидетельствует о том, что LoRA не просто экономит ресурсы, но и способствует более устойчивому и эффективному обучению, возможно, за счёт регуляризации.

### 3. Как Mamba показывает себя по сравнению с BERT?
- Слабые стороны:
  - Худшее качество среди всех моделей.
  - Высокие вычислительные затраты: обучение почти так же долгое, как у BERT, а инференс в 3 раза медленнее (250с с против ~85с с у BERT).
  - Большое число параметров без соответствующего прироста качества.
- Сильные стороны:
  - Теоретически Mamba эффективна для длинных последовательностей благодаря рекуррентной структуре, но в данной задаче это преимущество не реализовалось.
- Вывод: для данной задачи Mamba оказалась менее подходящей, чем BERT-архитектуры.

---

## Рекомендации по выбору архитектуры

Эксперименты показывают, что BERT с замороженными слоями обладает наилучшими практическими качествами, своеобразный трейдоф между скоростью и точностью ключевых метрик, но данный подход был проанализирован мной ввиду невозможности изначального запуска bert fine_tune на локальной машине, так что он рассматривается за скобками экспериментов.

LoRA — оптимальный компромисс между качеством, скоростью и эффективностью использования ресурсов в контексте поставленных ограничений по архитектурам в дз. Она не только радикально снижает вычислительную сложность, но и улучшает качество по сравнению с традиционным fine-tuning.  

Mamba, несмотря на теоретические преимущества для последовательностных данных, не оправдала ожиданий в рамках данной задачи классификации.  

Таким образом, для подобных NLP-задач (особенно при ограниченных ресурсах) рекомендуется использовать BERT в связке с LoRA, а не полный fine-tuning.